In [1]:
import os
import numpy as np
from scipy import misc
from shutil import make_archive
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.callbacks import LearningRateScheduler, CSVLogger

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from modis_utils.misc import restore_data
from modis_utils.model.loss_function import PSNRLoss, lossSSIM, SSIM, step_decay
from modis_utils.model.loss_function import mse_with_mask_tf, mse_with_mask_tf_1, mse_with_mask

# Parameters

In [3]:
config_path = 'config.dat'
config_params = restore_data(config_path)

In [4]:
lr = config_params['lr']

In [5]:
training = config_params['training']
crop_size = config_params['crop_size']
input_timesteps = config_params['input_timesteps']
output_timesteps = config_params['output_timesteps']
batch_size = config_params['batch_size']
compile_params = config_params['compile_params']
model_name = config_params['model_name']
preprocessed_type = config_params['preprocessed_type']
modis_product = config_params['modis_product']
monitor = config_params['monitor']
monitor_mode = config_params['monitor_mode']

raw_data_dir = config_params['raw_data_dir']
reservoir_index = config_params['reservoir_index']
used_band = config_params['used_band']
year_range = config_params['year_range']
model_keras = config_params['model_keras']
original_batch_size = config_params['original_batch_size']
resize_input = config_params['resize_input']

colab = config_params['colab']
TPU_FLAG = config_params['TPU_FLAG']

# End Parameters

In [6]:
from modis_utils.modis_utils import ModisUtils

In [7]:
modis_utils = ModisUtils(
    raw_data_dir=raw_data_dir,
    modis_product=modis_product,
    reservoir_index=reservoir_index,
    preprocessed_type=preprocessed_type,
    used_band=used_band,
    crop_size=crop_size,
    input_timesteps=input_timesteps,
    output_timesteps=output_timesteps,
    year_range=year_range,
    model_name=model_name,
    batch_size=batch_size,
    model_keras=model_keras,
    compile_params=compile_params,
    original_batch_size=original_batch_size,
    TPU_FLAG=TPU_FLAG,
    training=training,
    monitor=monitor,
    monitor_mode=monitor_mode,
    resize_input=resize_input)

In [8]:
modis_utils._preprocess_strategy_context.strategy.MAX

9996

In [9]:
modis_utils.create_water_cloud_mask()
modis_utils.make_archive_masked_data()

Not found band NDVI in 2000001 in ../raw_data/MOD13Q1/0/2000/2000001.
Not found band NDVI in 2000017 in ../raw_data/MOD13Q1/0/2000/2000017.
Not found band NDVI in 2000033 in ../raw_data/MOD13Q1/0/2000/2000033.


In [10]:
a = restore_data('masked_data/MOD13Q1/0/NDVI/2000/2000049/masked.dat')
a.shape

(511, 511)

In [11]:
modis_utils.preprocess_data()
modis_utils.make_archive_preprocessed_data()

Not found band NDVI in 2000001 in ../raw_data/MOD13Q1/0/2000/2000001.
Not found band NDVI in 2000017 in ../raw_data/MOD13Q1/0/2000/2000017.
Not found band NDVI in 2000033 in ../raw_data/MOD13Q1/0/2000/2000033.
Not found data 2000001 in preprocessed_data/MOD13Q1/0/NDVI/change_fill_value/2000/2000001.
Not found data 2000017 in preprocessed_data/MOD13Q1/0/NDVI/change_fill_value/2000/2000017.
Not found data 2000033 in preprocessed_data/MOD13Q1/0/NDVI/change_fill_value/2000/2000033.


In [12]:
a = restore_data(os.path.join(modis_utils._preprocessed_data_dir, '2000', '2000049', 'preprocessed.dat'))

In [13]:
a.shape

(511, 511)

In [14]:
a.max()

0.9610736017337668

In [9]:
modis_utils.create_data_file()

{'train': {'data': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/train_data.csv',
  'target': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/train_target.csv',
  'mask': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/train_mask.csv'},
 'val': {'data': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/val_data.csv',
  'target': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/val_target.csv',
  'mask': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/val_mask.csv'},
 'test': {'data': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/test_data.csv',
  'target': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/test_target.csv',
  'mask': 'data_file/preprocessed_data/MOD13Q1/0/NDVI/normalized_zero_one/12/1/test_mask.csv'}}

In [10]:
modis_utils.augment_data()

In [11]:
modis_utils.make_archive_augment_data()

In [12]:
a = restore_data(os.path.join(modis_utils._data_augment_merged_dir, 'train', '0.dat'))

In [13]:
a[0].shape, a[1].shape

((1024, 12, 63, 63, 1), (1024, 63, 63, 2))